<a href="https://colab.research.google.com/github/tranvietphi20104051/baitapAI/blob/main/phanidcardaligment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import torch


# Xoa cac box chong` len nhau (NMS)
def non_max_suppression_fast(boxes, labels, overlapThresh):
    # Tiếp theo ta thực hiện code xóa các box bị chồng lên nhau , và nếu không có box nào bị chồng lên nhau code sẽ trả về danh sách rỗng:
    if len(boxes) == 0:
        return []

    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    #
    # initialize the list of picked indexes
    pick = []

    #Lấy tọa độ của các box bị giới hạn:
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and sort the bounding
    # thiếp lập các hộp theo tọa độ y dưới cùng bên phải của các box bị giới hạn:
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # Lấy mục cuối cùng trong danh sách mục và thêm giá trị của mục đó cho danh sách các chứa mục đã chọn
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # Tìm tọa độ (x, y) lớn nhất khi bắt đầu, hộp giới hạn và tọa độ (x, y) nhỏ nhất cho phần cuối của hộp giới hạn:
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        #Tính toán chiều rộng và chiều cao cho hộp giới hạn:
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        #Tiếp theo ta sẽ tính toán tỉ lệ trùng lặp:
        overlap = (w * h) / area[idxs[:last]]

        #Xóa tất cả các mục khỏi danh sách chứa mục đó:
        idxs = np.delete(idxs, np.concatenate(([last],
                                               np.where(overlap > overlapThresh)[0])))

    # return only the bounding boxes that were picked using the
    #ta chỉ trả các hộp giới hạn đã được chọn sử dụng bằng kiểu dữ liệu số nguyên:

    final_labels = [labels[idx] for idx in pick]
    final_boxes = boxes[pick].astype("int")

    return final_boxes, final_labels


# Tim midpoint cua box
def get_center_point(tensor):
    x_mid = (tensor[0] + tensor[2]) / 2  # (Xmin + Xmax)/2
    y_mid = (tensor[1] + tensor[3]) / 2  # (Ymin + Ymax)/2
    point = [x_mid, y_mid]
    return point

def get_point(tensor, labels):
    x = 0
    y = 0
    if labels == 'top_left':
        x = tensor[0] # Xmin
        y = tensor[1] # Ymin
    if labels == 'top_right':
        x = tensor[2] # Xmax
        y = tensor[1] # Ymin
    if labels == 'bottom_left':
        x = tensor[0] # Xmin
        y = tensor[3] # Ymax
    if labels == 'bottom_right':
        x = tensor[2] # Xmax
        y = tensor[3] # Ymax

    point = [x, y]
    print(point)
    return point
#sẽ tìm lớp hành ảnh bị bỏ qua,
def find_miss_corner(coordinate_dict):
    position_name = ['top_left', 'top_right', 'bottom_left', 'bottom_right']
    position_index = np.array([0, 0, 0, 0])

    for name in coordinate_dict.keys():
        if name in position_name:
            position_index[position_name.index(name)] = 1

    index = np.argmin(position_index)

    return index


# Tim vi tri cua goc bi thieu
def calculate_missed_coord_corner(coordinate_dict):
    thresh = 0

    index = find_miss_corner(coordinate_dict)

    # Tính toán các tọa độ góc bị hụt:
    # case 1: missed corner is "top_left"
    if index == 0:
        midpoint = np.add(coordinate_dict['top_right'], coordinate_dict['bottom_left']) / 2
        y = 2 * midpoint[1] - coordinate_dict['bottom_right'][1] - thresh
        x = 2 * midpoint[0] - coordinate_dict['bottom_right'][0] - thresh
        coordinate_dict['top_left'] = (x, y)
    elif index == 1:  # "top_right"
        midpoint = np.add(coordinate_dict['top_left'], coordinate_dict['bottom_right']) / 2
        y = 2 * midpoint[1] - coordinate_dict['bottom_left'][1] - thresh
        x = 2 * midpoint[0] - coordinate_dict['bottom_left'][0] - thresh
        coordinate_dict['top_right'] = (x, y)
    elif index == 2:  # "bottom_left"
        midpoint = np.add(coordinate_dict['top_left'], coordinate_dict['bottom_right']) / 2
        y = 2 * midpoint[1] - coordinate_dict['top_right'][1] - thresh
        x = 2 * midpoint[0] - coordinate_dict['top_right'][0] - thresh
        coordinate_dict['bottom_left'] = (x, y)
    elif index == 3:  # "bottom_right"
        midpoint = np.add(coordinate_dict['bottom_left'], coordinate_dict['top_right']) / 2
        y = 2 * midpoint[1] - coordinate_dict['top_left'][1] - thresh
        x = 2 * midpoint[0] - coordinate_dict['top_left'][0] - thresh
        coordinate_dict['bottom_right'] = (x, y)

    return coordinate_dict


# chuyen doi ve goc chinh dien
def perspective_transoform(image, source_points):
    dest_points = np.float32([[0, 0], [500, 0], [500, 300], [0, 300]])
    M = cv2.getPerspectiveTransform(source_points, dest_points)
    dst = cv2.warpPerspective(image, M, (500, 300))

    return dst


# Crop Image (results, imgs) Tim anh crop trong 1 anh
# result = results.xyxy[i]; result_pandas = results.pandas().xyxy[i] ; img = imgs[i]
def CropImg(result, result_pandas, img):
    # 1. input overlap (Tensor 4*4, labels)
    # Tensor
    tensor = torch.tensor([])
    for i in range(len(result)):
        tensor_i = result[i][0:4].reshape(1, 4)
        tensor = torch.cat((tensor, tensor_i), 0)
    # Label
    label = list(result_pandas['name'])
    # 2. Overlap
    final_boxes, final_labels = non_max_suppression_fast(tensor.numpy(), label, 0.3)
    # 3. Tim midpoint cua cac boxes
    final_points = list(map(get_center_point, final_boxes))
    label_boxes = dict(zip(final_labels, final_points))
    # 3.5 Neu thieu 1 goc -> Noi suy goc con lai
    if (len(label_boxes) == 3):
        label_boxes = calculate_missed_coord_corner(label_boxes)
    elif (len(label_boxes) < 3):
        return None
    # 4. Crop anh
    source_points = np.float32(
        [label_boxes['top_left'], label_boxes['top_right'], label_boxes['bottom_right'], label_boxes['bottom_left']])
    crop = perspective_transoform(img, source_points)

    return crop


def preprocessing(path, model,init_size,debug=False):
    image = cv2.imread(path)[:,:,::-1]
    results_crop = model(image, size=init_size)
    # bbox_pandas = results_crop.pandas().xyxy
    if debug == True:
        for j in range(len(results_crop.xyxy)):
            img_crop = CropImg(results_crop.xyxy[j], results_crop.pandas().xyxy[j], image)
            # img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)
            return img_crop, image
    else:
        print('Error when starting ID card aligment')

# if __name__ == '__main__':
#     model_crop = torch.hub.load('ultralytics/yolov5', 'custom',
#                                 path='models/yolov5_l6_cccd.pt')
#     preprocessing('image/e6fa8a38302bf775ae3a.jpg',model_crop,init_size=1280, debug=True)



